# Introduction
This introduction describes PorePy's main concepts and features. For more detail on individual topics and code examples, we refer the reader to the corresponding tutorial notebooks, see the [readme](readme.md) for an overview.
For a more rigorous and theoretical description, please see [Keilegavlen et al.](https://link.springer.com/article/10.1007/s10596-020-10002-5) and references therein.

## Conceptual model
PorePy is a simulation toolbox covering several (coupled) physical processes tailored to _fractured_ porous media. 
The underlying mixed-dimensional model arises from dimension reduction of fractures, which is motivated by their high aspect ratio.
Averaging along the thickness of the fracture allows representing it as a two-dimensional subdomain with the aperture [m] accounting for the collapsed dimension.
Similarly, a fracture intersection is represented by a one-dimensional subdomain and intersections of fracture intersections by points.
To account for the collapsed area and volume, we introduce a _specific volume_ [m$^{3-d}$], with $d$ denoting the dimension of the subdomain in question.
Each lower-dimensional subdomain corresponds to an internal boundary for one or more higher-dimensional subdomain. 
Each such pair of neighboring subdomains exactly one dimension apart is connected through an _interface_. 
Thus, the entire mixed-dimensional domain can be represented by a graph with nodes corresponding to subdomains and edges corresponding to interfaces.

## Key components
The definition of a mixed-dimensional domain allows modeling, equation definition and numerical discretization to take place on individual subdomains and interfaces.
From the numerical point of view, this facilitates a high level of modularity and extensive code reuse. 
It also implies the need for a structure representing a mixed-dimensional grid and handling of this structure:
* The `GridBucket` object has a $d$-dimensional grid representing a subdomain and likewise an interface (or "mortar") grid at each edge. Each node and edge also has an associated data dictionary for storage of parameters, variables, discretization matrices etc.
* A `FractureNetwork` class processes a set of fractures (defined through vertices) by computing intersections to define the collection of subdomains. Then, the actual meshing of subdomains is done using [gmsh](https://gmsh.info/). The resulting mixed-dimensional mesh is conforming, in that any internal boundary corresponds to a face of the mesh, but may be non-matching, i.e. we do not require a one-to-one face correspondance between the two sides of a fracture.
* `DofManager` handles degree-of-freedom bookkeeping, including the mapping from local to global ordering, i.e. from subdomains to the whole mixed-dimensional domain.
* `EquationManager` takes care of assembly of the global equation system based on equations defined for (sets of) subdomains and interfaces.
* An automatic differentiation package underlies the `EquationManager`'s translation from user-defined equations to a linear system of equations.
* `Discretization` objects are defined on each subdomain and interface. Discretization is local to a subdomain and relies on the corresponding grid and parameters defined in the subdomain's data dictionary. To facilitate coupling over the interfaces, subdomain discretizations operate on grids and data of the two neighboring subdomains as well as those of the interface. 

## Capabilities
The above outlined framework enables implementation of numerical models for a wide range of phenomena in fractured porous media. The current capability includes
* Single-phase flow, both incompressible and compressible.
* Two-phase flow
* Linear elasticity
* Fracture contact mechanics
* Flow and transport, either decoupled, weakly coupled or strongly coupled.
* Biot poroelasticity
Combinations of the above are also possible, e.g. solving the Biot equations in the matrix, and flow and contact mechanics in fractures. 

The setup of a coupled multi-physics mixed-dimensional simulation is nontrivial. We therefore provide `Model` classes with (unitary) parameters, discretizations, equation definition etc, which all may be modularly adjusted according to the user's needs. Several of the models are demonstrated in other tutorials. 